In [1]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import scipy.stats as stats

In [2]:
#Mercado local
Tickers=['CEMEXCPO.MX','CUERVO.MX','LACOMERUBC.MX','GRUMAB.MX','VOLARA.MX']
Titulos=[800,200,250,100,700]

In [3]:
#Extracción de precios de Yahoo Finance
def get_data(symbol:str):
    data=yf.download(tickers=symbol,period='253d',interval='1d')
    data.reset_index(inplace=True)
    return data

In [4]:
dataframes=[] #lista que contendrá dataframes como elementos
df_final=pd.DataFrame()

for Ticker in Tickers:
    data=get_data(Ticker)
    #Renombrar las columnas por cada ticker
    data=data.set_axis(['Date',f'Close_{Ticker}','High','Low','Open','Volume'],axis=1)

    if df_final.empty:
        df_final=data[['Date',f'Close_{Ticker}']]
    else:
        df_final=pd.merge(df_final,data[['Date',f'Close_{Ticker}']],on='Date',how='outer')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
df_final

,Date,Close_CEMEXCPO.MX,Close_CUERVO.MX,Close_LACOMERUBC.MX,Close_GRUMAB.MX,Close_VOLARA.MX
0,2024-04-10,14.12,33.840218,36.588711,313.615784,12.66
1,2024-04-11,14.35,33.563972,36.737606,315.577271,12.34
2,2024-04-12,14.26,34.096733,36.003056,311.930328,11.74
3,2024-04-15,14.06,33.267994,35.427322,311.841644,12.04
4,2024-04-16,14.35,32.064346,35.069973,313.497528,12.03
...,...,...,...,...,...,...
248,2025-04-04,10.47,19.990000,35.549999,361.670013,9.96
249,2025-04-07,10.64,19.510000,35.799999,359.000000,10.20
250,2025-04-08,10.61,19.850000,36.490002,366.989990,9.84
251,2025-04-09,10.88,19.950001,36.959999,365.230011,10.37


In [6]:
#Calcular los rendimientos de cada activo, con los precios de cierre del dataframe
for Ticker in Tickers:
    df_final[f'Rend_{Ticker}']=df_final[f'Close_{Ticker}']/df_final[f'Close_{Ticker}'].shift(1)-1 #Shift toma el dato anterior

In [7]:
# Calcular los Precios Simulados con la información histórica
for Ticker in Tickers:
    #Obtener el último precio para cada Acción
    ultimo_precio=df_final[f'Close_{Ticker}'].iloc[-1]
    #Calcular los Precios Simulados con la información histórica
    df_final[f'PrecioSim_{Ticker}']=ultimo_precio*(1+df_final[f'Rend_{Ticker}'])

df_final

,Date,Close_CEMEXCPO.MX,Close_CUERVO.MX,Close_LACOMERUBC.MX,Close_GRUMAB.MX,Close_VOLARA.MX,Rend_CEMEXCPO.MX,Rend_CUERVO.MX,Rend_LACOMERUBC.MX,Rend_GRUMAB.MX,Rend_VOLARA.MX,PrecioSim_CEMEXCPO.MX,PrecioSim_CUERVO.MX,PrecioSim_LACOMERUBC.MX,PrecioSim_GRUMAB.MX,PrecioSim_VOLARA.MX
0,2024-04-10,14.12,33.840218,36.588711,313.615784,12.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-11,14.35,33.563972,36.737606,315.577271,12.34,0.016289,-0.008163,0.004069,0.006254,-0.025276,10.477940,19.985511,36.166581,369.174629,9.406082
2,2024-04-12,14.26,34.096733,36.003056,311.930328,11.74,-0.006272,0.015873,-0.019995,-0.011556,-0.048622,10.245338,20.469840,35.299798,362.640187,9.180794
3,2024-04-15,14.06,33.267994,35.427322,311.841644,12.04,-0.014025,-0.024306,-0.015991,-0.000284,0.025554,10.165400,19.660243,35.443996,366.775698,9.896593
4,2024-04-16,14.35,32.064346,35.069973,313.497528,12.03,0.020626,-0.036180,-0.010087,0.005310,-0.000831,10.522653,19.420966,35.656673,368.828143,9.641984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2025-04-04,10.47,19.990000,35.549999,361.670013,9.96,-0.057606,-0.068933,-0.013870,-0.050735,-0.055924,9.716086,18.760992,35.520417,348.266394,9.110331
249,2025-04-07,10.64,19.510000,35.799999,359.000000,10.20,0.016237,-0.024012,0.007032,-0.007382,0.024096,10.477403,19.666158,36.273306,364.171529,9.882530
250,2025-04-08,10.61,19.850000,36.490002,366.989990,9.84,-0.002820,0.017427,0.019274,0.022256,-0.035294,10.280930,20.501153,36.714243,375.045374,9.309412
251,2025-04-09,10.88,19.950001,36.959999,365.230011,10.37,0.025448,0.005038,0.012880,-0.004796,0.053862,10.572367,20.251511,36.483944,365.120553,10.169766


In [8]:
#Calcular las Pérdidas y Ganancias para cada Activo y las del Portafolio
i=0
df_final['Portafolio']=0
for Ticker in Tickers:
    #Obtener el último precio para cada Acción
    ultimo_precio=df_final[f'Close_{Ticker}'].iloc[-1]
    df_final[f'Perd_Gan_{Ticker}']=(df_final[f'PrecioSim_{Ticker}']-ultimo_precio)*Titulos[i]
    df_final['Portafolio']=df_final['Portafolio']+df_final[f'Perd_Gan_{Ticker}']
    i=i+1

df_final

,Date,Close_CEMEXCPO.MX,Close_CUERVO.MX,Close_LACOMERUBC.MX,Close_GRUMAB.MX,Close_VOLARA.MX,Rend_CEMEXCPO.MX,Rend_CUERVO.MX,Rend_LACOMERUBC.MX,Rend_GRUMAB.MX,...,PrecioSim_CUERVO.MX,PrecioSim_LACOMERUBC.MX,PrecioSim_GRUMAB.MX,PrecioSim_VOLARA.MX,Portafolio,Perd_Gan_CEMEXCPO.MX,Perd_Gan_CUERVO.MX,Perd_Gan_LACOMERUBC.MX,Perd_Gan_GRUMAB.MX,Perd_Gan_VOLARA.MX
0,2024-04-10,14.12,33.840218,36.588711,313.615784,12.66,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-11,14.35,33.563972,36.737606,315.577271,12.34,0.016289,-0.008163,0.004069,0.006254,...,19.985511,36.166581,369.174629,9.406082,196.819100,134.351571,-32.897773,36.645234,229.462396,-170.742329
2,2024-04-12,14.26,34.096733,36.003056,311.930328,11.74,-0.006272,0.015873,-0.019995,-0.011556,...,20.469840,35.299798,362.640187,9.180794,-920.238193,-51.729705,63.968151,-180.050576,-423.981787,-328.444276
3,2024-04-15,14.06,33.267994,35.427322,311.841644,12.04,-0.014025,-0.024306,-0.015991,-0.000284,...,19.660243,35.443996,366.775698,9.896593,-195.448009,-115.680117,-97.951284,-144.001039,-10.430668,172.615098
4,2024-04-16,14.35,32.064346,35.069973,313.497528,12.03,0.020626,-0.036180,-0.010087,0.005310,...,19.420966,35.656673,368.828143,9.641984,122.686838,170.122312,-145.806802,-90.831910,194.813831,-5.610593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2025-04-04,10.47,19.990000,35.549999,361.670013,9.96,-0.057606,-0.068933,-0.013870,-0.050735,...,18.760992,35.520417,348.266394,9.110331,-3116.958267,-475.131894,-277.801501,-124.895982,-1861.361041,-377.767848
249,2025-04-07,10.64,19.510000,35.799999,359.000000,10.20,0.016237,-0.024012,0.007032,-0.007382,...,19.666158,36.273306,364.171529,9.882530,-7.596920,133.921743,-96.768291,63.326303,-270.847597,162.770922
250,2025-04-08,10.61,19.850000,36.490002,366.989990,9.84,-0.002820,0.017427,0.019274,0.022256,...,20.501153,36.714243,375.045374,9.309412,798.660604,-23.256172,70.230680,173.560676,816.536952,-238.411532
251,2025-04-09,10.88,19.950001,36.959999,365.230011,10.37,0.025448,0.005038,0.012880,-0.004796,...,20.251511,36.483944,365.120553,10.169766,534.072197,209.892925,20.302344,115.985928,-175.945179,363.836179


In [9]:
VaR_99=df_final['Portafolio'].quantile(0.01)
VaR_99

-2436.4070766367886

In [15]:
dfpl=df_final[:]
#Calcular los parámetros de la dist Normal Teórica
media=dfpl['Portafolio'].mean()
volatilidad=dfpl['Portafolio'].std()

#Crear el prefijo para mi gráfico
fig=go.Figure()

#Gráfico
fig.add_trace(go.Histogram(x=dfpl['Portafolio'],
                           nbinsx=50, #nbins es el nro. de barritas
                           marker_color='skyred',
                           opacity=0.75,name='Datos Reales'))

#Generar la dist normal teórica mediante generar un rango en el eje X
#Tal rango va de la pérdida máxima a la menor y lo particiona en 100
#Después grafica esos valores al evaluarlos en una N(media,volatilidad)
x=np.linspace(dfpl['Portafolio'].min(),dfpl['Portafolio'].max(),100)
y=stats.norm.pdf(x,media,volatilidad)

#Escalamiento para la función de distribución teórica 
bin_pdf=(dfpl['Portafolio'].max()-dfpl['Portafolio'].min())/50
y_scaled=y*bin_pdf*len(dfpl)

#Añadirla al gráfico anterior de barras
fig.add_trace(go.Scatter(x=x,
                         y=y_scaled,
                         mode='lines',
                         name='Distribución Normal',
                         line=dict(color='green',width=2,dash='dash')))

#Añadir una línea verticar que señale el VaR
fig.add_shape(go.layout.Shape(type='line',
                              x0=VaR_99,
                              y0=0,
                              x1=VaR_99,
                              y1=max(y_scaled),
                              line=dict(color='red',width=3,dash='dash')))

#Mostrar Gráfico
fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of histogram.marker
        Received value: 'skyred'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deeps